In [1]:
from importlib import reload 
import sys
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr, kendalltau
import plotly.express as px
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.spatial.distance import braycurtis
from scipy.special import softmax
import seaborn as sns
# # adding Folder_2 to the system path
# sys.path.insert(0, r'C:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\code')
# # sys.path.insert(0, r'C:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\code')

from imports import  *

import model_class
reload(model_class)


<module 'model_class' from '/home/jupyter-tomeroron/baboons/model_class.py'>

$$f(D,\Delta_t) = \alpha e^{-\lambda \Delta_t} \cdot \cos\left(\frac{2 \pi \Delta_t}{365}\right) D_{t-1} + \left(1 - \alpha e^{-\lambda \Delta_t} \cdot \cos\left(\frac{2 \pi \Delta_t}{365}\right)\right) \cdot \overline{\{D_1,...,D_{t-2}\}}$$

In [2]:
import model_class
reload(model_class)


<module 'model_class' from '/home/jupyter-tomeroron/baboons/model_class.py'>

In [3]:
data_path = r"train_data.csv"
metadata_path = r"train_metadata.csv"

In [4]:
metadata_df = pd.read_csv(metadata_path)

In [5]:
metadata_df.columns

Index(['sample', 'baboon_id', 'collection_date', 'sex', 'age', 'social_group',
       'group_size', 'rain_month_mm', 'season', 'hydro_year', 'month',
       'diet_PC1', 'diet_PC2', 'diet_PC3', 'diet_PC4', 'diet_PC5', 'diet_PC6',
       'diet_PC7', 'diet_PC8', 'diet_PC9', 'diet_PC10', 'diet_PC11',
       'diet_PC12', 'diet_PC13'],
      dtype='object')

In [6]:
metadata_df.social_group.unique()

array(['g_1.22', 'g_2.2', 'g_1.1', 'g_1.12', 'g_1.21', 'g_1.211',
       'g_1.11', 'g_2.1', 'g_2.12', 'g_2.11'], dtype=object)

In [7]:
# data_df = pd.read_csv(r"C:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\train_data.csv", index_col="sample")
# metadata_df = pd.read_csv(r"C:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\train_metadata.csv",  index_col="sample")

In [8]:
super_model = model_class.superModel(data_path, metadata_path)

In [19]:
import time
start = time.time()
super_model.fit()
time.time() - start

for lambda = 0.0 the objective function is 0.3670651575037061
for lambda = 1e-08 the objective function is 0.3670651575037061
for lambda = 0.0 the objective function is 0.3670651575037061


10940.704362630844

In [10]:
# for baboon in super_model.baboons:
#     baboon.alpha_ = np.zeros([61,61])
#     baboon.beta_ = np.eye(61,61)



In [11]:
baboon_101 = super_model.baboons[0]

In [12]:
baboon_101.fit(0)

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[ 9.99560751e-01, -7.52734711e-05, -6.95880161e-05, ...,
         -2.14871154e-05, -1.26788573e-04, -5.54778395e-05],
        [-2.42678641e-04,  9.99945119e-01, -1.70861341e-05, ...,
         -1.49847119e-05, -6.01530891e-05, -4.34695731e-05],
        [-7.14703165e-05, -7.50187236e-05,  9.99941819e-01, ...,
         -1.52833813e-05, -1.21462790e-04, -7.36542145e-05],
        ...,
        [-2.84972081e-04, -5.47160373e-05, -1.75448747e-05, ...,
          9.99983832e-01, -6.24160466e-05, -4.34661569e-05],
        [-2.67673170e-04, -5.78305930e-05, -4.19201036e-05, ...,
         -1.54009945e-05,  9.99908403e-01, -2.34738525e-05],
        [-1.73669419e-04, -4.10358079e-05, -2.36890702e-05, ...,
         -1.13089311e-05, -6.84743502e-05, 

In [13]:
# calculate the objective function

'''for the ith row in self.data (from row 3)
1. calculate mean of previous i-2 rows = d_mean
2. calculate time difference between ith row and i-1th row = d_time
calculate the prediction for the ith row using the formula'''


'''
calculate difference between prediction and actual value using bray-curtis dissimilarity and return the sum/mean -TBD'''
self = baboon_101
lambda_ = 0
method="counts"
alpha = baboon_101.alpha_
beta = baboon_101.beta_


D_t1 = self.transformed_data[1:-1].values
D_mean = self.df_cumulative_mean[:-2].values
cos = 0# np.cos((2*np.pi*self.delta_t[2:].values)/365)
exp = np.exp(-lambda_*self.delta_t[2:].values)

f = alpha@(exp*cos*D_t1.T) + beta@((1-exp*cos)*D_mean.T)

f = to_composition(f.T, type = method) # transpose f to match the shape of D - each row is a sample

# calculate bray-curtis dissimilarity
bc =  np.array([braycurtis(self.data.values[i+2], f[i]) for i in range(len(f))])

f.shape

(51, 61)